# 1. 

### GPU사용, 하이퍼파라미터 선언

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

# 2.

### dataloader : 데이터를 불러 들어와서 옵션에 맞게 batchsize로 슬라이싱 해 mini-batchs로 만들어준다

In [ ]:
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() # 데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# 3.
### input size를 알기 위해서 

In [ ]:
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

# 4. 
### model 선언

In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input_channels = 1, out_channels = 10, kernel size = 5, stribe = 1, padding= 0
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # input_channels = 10, out_channels = 20, kernel size = 5, stribe = 1, padding= 0
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.mp = nn.MaxPool2d(2) 
        self.fc1 = nn.Linear(320,100) #in_features=320, out_features=100, bias=True
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) #view(x.size(0),-1)을 해주는 이유: linear layer에 넣어주기 위해서는 shape 변형을 해줘야하기 때문
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) 

# 5.
### define a loss function, select optimizer

In [ ]:
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device) # 다중분류를 위해 손실함수로는 crossentropy를 사용해준다
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# 6.
### train data로 모델 훈련

In [ ]:
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        # 모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : 학습을 할 때마다 gradient를 더해줘야하기 때문에 학습이 끝나면
        # 0으로 초기화 해줘야한다. 
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        #gradient를 계산할수있는 자동 미분 기능이다. 계산된 gradient는 자동 축적된다
        optimizer.step()
        #계산된 gradient를 바탕으로 파라미터를 업데이트 해준다 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 7. 

### test 데이터로 accuracy 평가

In [ ]:
model.eval()
#model.eval : evaluation과정에서 사용하지 않을 layer들의 전원을 끈다.
with torch.no_grad(): 
    #no_grad 사용 이유: test데이터에 데에서는 backprogation을 진행하지 않으므로 자동으로 gradient를 트래킹하지 않도록 하는 것
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        #torch.max(out.data)를 하면 최대값, 최대값의 위치가 두개가 나오는데 거기서 최대값의 위치만 사용하기 위해 [1]을 해준다
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')